In [ ]:
!pip install openai
!pip install langchain
!pip install langchain_community
!pip install langchain_openai
!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# loading from a .env file
# load_dotenv(dotenv_path="/full/path/to/your/.env")

# or 
# if you're on google colab just uncomment below and replace with your openai api key
# os.environ["OPENAI_API_KEY"] = "<your-openai-api-key>"

# Notes Summarizer App

In [1]:
from openai import OpenAI

def get_response(prompt):
    client = OpenAI()
    response = client.chat.completions.create(model="gpt-4o", 
                             messages=
                             [
                                 {"role": "system", "content": "You are a helpful research assistant."},
                                 {"role": "user", "content": prompt}   
                             ],
                             temperature=0.0,
                             )
    return response.choices[0].message.content

notes_path = "./Reinventing-explanation.txt"
with open(notes_path, "r") as f:
    text = f.read()

get_response(f"Summarize this text: {text}")

'Michael Nielsen\'s essay "Reinventing Explanation" explores the potential of non-traditional media as powerful tools for deeper explanations, particularly in scientific and mathematical contexts. Nielsen argues that such media should not be seen merely as vehicles for popularization or education but as opportunities to create more profound and nuanced understandings.\n\nHe uses the example of the London Underground map to illustrate how certain representations can become internalized, transforming the way we think about complex systems. Similarly, he discusses how a new vocabulary of operations for difference equations can change our approach to mathematical thinking.\n\nNielsen envisions a future where these dynamic problem-solving mediums expand our cognitive capabilities, especially in mathematics. He focuses on designing media that explain scientific ideas more effectively, suggesting that we currently lack a basic vocabulary for digital explanations. He believes that developing p

Ok , great we can get a basic summary easy using just ChatGPT, but how about if we want to be able to specify different types of summaries? 
What if the contents we want to summarize are bigger than the context length of our model?

Let's address these issues by using langchain for summarization.

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import TextLoader
from IPython.display import Markdown, display

With langchain, summarization is as simple as this:

In [4]:
docs = TextLoader(notes_path).load()
llm = ChatOpenAI()
chain = load_summarize_chain(llm, chain_type="stuff") #stuff means everything goes inside the model's prompt
chain.invoke(docs)

{'input_documents': [Document(metadata={'source': './Reinventing-explanation.txt'}, page_content='[source](https://michaelnielsen.org/reinventing_explanation/index.html)\nMedia for thought. #intelligence #learning #problemsolving \n[[A dynamic problem solving medium]]\n\nMiachel Nielsen in his [[Reinventing-explanation]], says:\n\n> I believe it\'s worth taking non-traditional media seriously not just as a vehicle for popularization or education, which is how they are often viewed, but as an opportunity for explanations which can be, in important ways, deeper. Nielsen, [[Reinventing-explanation]], conclusion\n\n> This agglomeration of ideas has turned maps into a powerful _medium for thought_. Consider the famous map of the London Underground, excerpted here: \n\n> we internalize the map: the representations used in the map become the representations we use to think about the Underground. This is the sense in which this map is a medium for thought.\n\n> He\'s created a vocabulary of op

But as always, we would like to go beyond, why not set up a prompt template to allow for some customization of the summaries?

In [9]:
prompt_template = """Write a concise summary of the following text:

{text}

The summary should be in this style: {style}.
Let's think step by step.

SUMMARY:
"""

prompt_summarization = ChatPromptTemplate.from_template(prompt_template)
chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt_summarization)
summary = chain.invoke({"input_documents": docs, "style": "concise"})
Markdown(summary)

AttributeError: 'str' object has no attribute 'page_content'

Ok, let's wrap this into a streamlit app to allow for a nice looking user interface.

[Notes summarize example](https://platform.openai.com/examples/default-meeting-notes-summarizer)